In [6]:
import pandas as pd
import numpy as np
import matplotlib as mp
import datetime as dt

%matplotlib inline

Tiempos hasta Reaparicion
=====================

Se desea armar un set de entrenamiento con el tiempo que pasa hasta que un un dispositivo convierta. 

Sabiendo que el set de datos posee informacion del dia 18 al 26 

Para esto se arman ventanas de los siguientes dias: 

1. 21 al 23
2. 22 al 24
3. 23 al 25
4. 24 al 26

Y se desea obtener el minimo de tiempo en segundos hasta que instala cada dispositivo.


Luego, se cruza los labels armados con los labels a predecir y se  asigna un valor por defecto para aquellos que no se encuentran en alguna ventana.

Opciones: 

- Maximo tiempo posible: Tiempo de ventana
- Maximo tiempo visto: Maximo tiempo hasta install de alguna ventana
- Media de tiempo entre installs vistos

Para los ultimos 2 hay que definir que hacer si no se instalo ninguna vez. 

In [2]:
#Optimizado para menos memoria
auction_dtypes = {
    'device_id': np.int64,
    'ref_type_id': np.int8,
    'source_id': np.int8
}

auctions = pd.read_csv('../data/auctions.csv.gzip',
                       compression = 'gzip',
                       dtype = auction_dtypes,
                       parse_dates = ['date'])
auctions.columns = ['date','ref_hash','ref_type_id','source_id']
to_predict = pd.read_csv('../data/target_final_competencia_revamped.csv')
to_predict = to_predict.apply(lambda x: np.int64(x['ref_hash'][0:x['ref_hash'].find('_')]), axis='columns').drop_duplicates().to_frame()
to_predict.columns = ['ref_hash']

In [4]:
all_ref_hash = auctions['ref_hash'].drop_duplicates().to_frame()

In [7]:
window = 3
j=1
auctions_training = pd.DataFrame()

for i in range(21,25): #[19, 20, 21, 22, 23, 24]
    print("Ventana: ", j)
    temp = pd.DataFrame()
    
    temp[['ref_hash','date_appear']] = auctions.loc[(auctions['date'].dt.day >= i) & (auctions['date'].dt.day < (i+window))].groupby('ref_hash', as_index=False)['date'].min()
    temp['window_nr'] = j
    temp['window_date_start'] = dt.datetime(2019,4,i)
    temp['secs_to_appear'] = (temp['date_appear']-temp['window_date_start']).transform(lambda x: x.total_seconds())
    print("  Encontrados: ", len(temp))
    auctions_training = auctions_training.append(temp,sort=True)
    
    #Agrego los que no se encontraron en esta ventana pero si estan en installs (aparecen en alguna)
    temp = all_ref_hash.merge(auctions_training.loc[auctions_training['window_nr'] == j], how='left',on='ref_hash')
    temp = temp.loc[temp['window_nr'].isnull()]
    temp['window_nr'] = j
    temp['window_date_start'] = dt.datetime(2019,4,i)
    temp['date_appear'] = dt.datetime(2019,4,i+window)
    temp['secs_to_appear'] = (temp['date_appear']-temp['window_date_start']).transform(lambda x: x.total_seconds())
    print("  Agregados sin aparecer en esta ventana: ", len(temp))
    auctions_training = auctions_training.append(temp[['ref_hash','window_nr','window_date_start','date_appear','secs_to_appear']],sort=True)
    
    #Agrego los que no se encontraron en install pero se deben predecir conversiones
    temp = to_predict.merge(auctions_training.loc[auctions_training['window_nr'] == j], how='left', on='ref_hash')
    temp = temp.loc[temp['window_nr'].isnull()]
    print("  Agregados sin aparecer pero en lista a predecir: ", len(temp))
    temp['window_nr'] = j
    temp['window_date_start'] = dt.datetime(2019,4,i)
    temp['date_appear'] = dt.datetime(2019,4,i+window)
    temp['secs_to_appear'] = (temp['date_appear']-temp['window_date_start']).transform(lambda x: x.total_seconds())
    auctions_training = auctions_training.append(temp,sort=True)
    
    j = j+1

sorted_auctions = auctions_training.sort_values(['ref_hash','window_nr'])[['ref_hash','window_nr','window_date_start','date_appear','secs_to_appear']]

Ventana:  1
  Encontrados:  326945
  Agregados sin aparecer en esta ventana:  148393
  Agregados sin aparecer pero en lista a predecir:  2944
Ventana:  2
  Encontrados:  325310
  Agregados sin aparecer en esta ventana:  150028
  Agregados sin aparecer pero en lista a predecir:  2944
Ventana:  3
  Encontrados:  322403
  Agregados sin aparecer en esta ventana:  152935
  Agregados sin aparecer pero en lista a predecir:  2944
Ventana:  4
  Encontrados:  329779
  Agregados sin aparecer en esta ventana:  145559
  Agregados sin aparecer pero en lista a predecir:  2944


In [8]:
sorted_auctions['window_nr'].value_counts()

4    478282
3    478282
2    478282
1    478282
Name: window_nr, dtype: int64

In [9]:
sorted_auctions.head(20)

,ref_hash,window_nr,window_date_start,date_appear,secs_to_appear
0,40621409780134,1,2019-04-21,2019-04-23 15:00:57.222979,226857.222979
0,40621409780134,2,2019-04-22,2019-04-23 15:00:57.222979,140457.222979
0,40621409780134,3,2019-04-23,2019-04-23 15:00:57.222979,54057.222979
461695,40621409780134,4,2019-04-24,2019-04-27 00:00:00.000000,259200.000000
392140,41863526108385,1,2019-04-21,2019-04-24 00:00:00.000000,259200.000000
392140,41863526108385,2,2019-04-22,2019-04-25 00:00:00.000000,259200.000000
392140,41863526108385,3,2019-04-23,2019-04-26 00:00:00.000000,259200.000000
392140,41863526108385,4,2019-04-24,2019-04-27 00:00:00.000000,259200.000000
361910,69039685746313,1,2019-04-21,2019-04-24 00:00:00.000000,259200.000000
361910,69039685746313,2,2019-04-22,2019-04-25 00:00:00.000000,259200.000000


In [10]:
sorted_auctions.to_csv('auctions_training.csv',index=False)